# Kotlin **Asynchronous/Concurren** Programming **Best Practises**

- **NEVER** make your code complicated. The previous sections (11-14) were prepared solely for demonstration and explanation purposes. We must follow best practices and implement our logic in the clearest way possible. Always aim to design your code in the most straightforward way; otherwise, debugging and code modifications will become difficult.
- Be sure to consult other **resources** and **communities** as well.
- Remember: **DYE** – Do Your Experiment. Testing and experimenting yourself is key to truly understanding and mastering the concepts.

In [12]:
// Importing some neccessary libs in jupyter notebook
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.5.2")
import kotlinx.coroutines.*
import java.util.concurrent.Executors
import kotlin.coroutines.CoroutineContext

In [2]:
// Use 'runBlocking' only at the entry of your program.
fun main() = runBlocking {
    println("Hi")
}

In [3]:
// Try using 'suspend' functions to better organise your code.

suspend fun fetchBackend() {
    // Fetch your backend
    println("fetched")
}
fun main() = runBlocking {
    fetchBackend()
}
main()

fetched


In [4]:
// Use 'coroutineScope' if you wanna 'launch' multiple requests in a specific scope.
suspend fun fetchBackend() = coroutineScope {
    // Fetch your backend
    println("fetching started.")
    List(5) { i ->
        launch { println("end-point $i fetched") }
    }
    
}
fun main() = runBlocking {
    fetchBackend()
}
main()

fetching started.
end-point 0 fetched
end-point 1 fetched
end-point 2 fetched
end-point 3 fetched
end-point 4 fetched


[StandaloneCoroutine{Completed}@77169011, StandaloneCoroutine{Completed}@7dad756d, StandaloneCoroutine{Completed}@12f8dea4, StandaloneCoroutine{Completed}@5c280955, StandaloneCoroutine{Completed}@1f7d8cd9]

In [5]:
/*
Use 'coroutineScope', 'async' and 'awaitAll' if you wanna launch multiple requests
and get the result.
You can also use 'launch' and 'join/joinAll'. 
*/

suspend fun fetchBackend() = coroutineScope {
    // Fetch your backend
    println("fetching started.")
    val reqs = List(5) { i ->
        async { 
            println("end-point $i fetched") 
            i*i
        }
    }
    val res = reqs.awaitAll()
    println("fetching finished.")
    return@coroutineScope res
}
fun main() = runBlocking {
    val res = fetchBackend()
    println("fetch result: $res")
}
main()

fetching started.
end-point 0 fetched
end-point 1 fetched
end-point 2 fetched
end-point 3 fetched
end-point 4 fetched
fetching finished.
fetch result: [0, 1, 4, 9, 16]


In [6]:
/* 
Use 'withContext' to create a coroutine scope 
which is run on the given dispatcher.
*/

suspend fun fetchBackend() = withContext(Dispatchers.IO) {
    // Fetch your backend on IO threads pool
    println("fetching started.")
    val reqs = List(5) { i ->
        async { 
            println("end-point $i fetched") 
            i*i
        }
    }
    val res = reqs.awaitAll()
    println("fetching finished.")
    return@withContext res
}
fun main() = runBlocking {
    val res = fetchBackend()
    println("fetch result: $res")
}
main()

fetching started.
end-point 0 fetched
end-point 2 fetched
end-point 3 fetched
end-point 1 fetched
end-point 4 fetched
fetching finished.
fetch result: [0, 1, 4, 9, 16]


In [7]:
// Only create custom 'CoroutineScope' when it is neccessary.

val threadNo = 10
val contextJob = Job()
val contextDispatcher = Executors.newFixedThreadPool(threadNo).asCoroutineDispatcher()
val context: CoroutineContext = contextDispatcher + contextJob + CoroutineName("Powerful but heavy Scope")
val myScope = CoroutineScope(context)

suspend fun doImportant(): List<Int> {
    // Fetch your backend
    println("Doing important started.")
    val reqs = List(100) { i ->
        myScope.async { 
            delay(1000)
            print("imp{$i} done, ") 
            i*i
        }
    }
    val res = reqs.awaitAll()
    println("\nDoing important finished.")
    return res
}
fun main() = runBlocking {
    val res = doImportant()
    myScope.cancel()
    println("Doing important result: ${res.sum()}")
}
main()                           

Doing important started.
imp{9} done, imp{4} done, imp{5} done, imp{8} done, imp{7} done, imp{6} done, imp{3} done, imp{0} done, imp{1} done, imp{2} done, imp{10} done, imp{11} done, imp{12} done, imp{13} done, imp{14} done, imp{15} done, imp{16} done, imp{17} done, imp{18} done, imp{19} done, imp{20} done, imp{21} done, imp{22} done, imp{23} done, imp{28} done, imp{24} done, imp{25} done, imp{26} done, imp{27} done, imp{29} done, imp{33} done, imp{30} done, imp{31} done, imp{34} done, imp{32} done, imp{35} done, imp{36} done, imp{37} done, imp{38} done, imp{41} done, imp{39} done, imp{40} done, imp{42} done, imp{43} done, imp{44} done, imp{45} done, imp{46} done, imp{47} done, imp{48} done, imp{50} done, imp{52} done, imp{49} done, imp{51} done, imp{53} done, imp{54} done, imp{56} done, imp{55} done, imp{57} done, imp{60} done, imp{58} done, imp{59} done, imp{62} done, imp{61} done, imp{63} done, imp{64} done, imp{65} done, imp{67} done, imp{68} done, imp{72} done, imp{69} done, imp{7

In [8]:
// Or it is better to do it using 'withContext' as:

val threadNo = 10
val myDispatcher: CoroutineContext = Executors.newFixedThreadPool(threadNo).asCoroutineDispatcher()

suspend fun doImportant(): List<Int> = withContext(myDispatcher) {
    // Fetch your backend
    println("Doing important started.")
    val reqs = List(100) { i ->
        async { 
            delay(1000)
            print("imp{$i} done, ") 
            i*i
        }
    }
    val res = reqs.awaitAll()
    println("\nDoing important finished.")
    return@withContext res
}
fun main() = runBlocking {
    val res = doImportant()
    myDispatcher.cancel()
    println("Doing important result: ${res.sum()}")
}
main()       

Doing important started.
imp{1} done, imp{0} done, imp{3} done, imp{4} done, imp{5} done, imp{6} done, imp{7} done, imp{9} done, imp{10} done, imp{11} done, imp{12} done, imp{2} done, imp{13} done, imp{14} done, imp{17} done, imp{15} done, imp{18} done, imp{16} done, imp{19} done, imp{20} done, imp{21} done, imp{22} done, imp{23} done, imp{24} done, imp{25} done, imp{26} done, imp{27} done, imp{28} done, imp{29} done, imp{30} done, imp{31} done, imp{33} done, imp{32} done, imp{34} done, imp{35} done, imp{37} done, imp{36} done, imp{38} done, imp{43} done, imp{39} done, imp{40} done, imp{41} done, imp{42} done, imp{44} done, imp{45} done, imp{46} done, imp{47} done, imp{48} done, imp{8} done, imp{49} done, imp{50} done, imp{51} done, imp{52} done, imp{53} done, imp{54} done, imp{55} done, imp{56} done, imp{57} done, imp{58} done, imp{59} done, imp{60} done, imp{61} done, imp{62} done, imp{63} done, imp{64} done, imp{65} done, imp{67} done, imp{66} done, imp{68} done, imp{69} done, imp{7

In [9]:
/*
Consider finalising things in critical part of your code
when a cancellation request might be received.
*/

fun main() = runBlocking {
    launch {
        withTimeoutOrNull(3000) {
            println("Program Started")
            delay(1000)
            criticalPart() 
            println("Program Finished")
        } 
    }
}

suspend fun criticalPart() {
    try {
        println("Critical part of the code started.")
        delay(3000)
        println("Critical part of the code finished.")
    } finally {
        println("Critical part of the code encountered an exception.")
        println("Finalization has been done.")
    }
    
}
main()

Program Started
Critical part of the code started.
Critical part of the code encountered an exception.
Finalization has been done.


StandaloneCoroutine{Completed}@631f0cf3

In [10]:
/*
You can catch a cancellation error (and re-throw it if you want)
*/

fun main() = runBlocking {
    launch {
        withTimeoutOrNull(3000) {
            println("Program Started")
            delay(1000)
            criticalPart() 
            println("Program Finished")
        } 
    }
}

suspend fun criticalPart() {
    try {
        println("Critical part of the code started.")
        delay(3000)
    } catch(e: CancellationException) {
        println("A cancellation request was received: ${e.message}")
    } catch(e: Exception) {
        println("Other exceptions happened: ${e.message}")
    } finally {
        println("Critical part of the code encountered an exception.")
        println("Finalization has been done.")
    }
    delay(1000) // Cancellation request still exists and will apply here at this suspension point.
    println("Critical part of the code finished.")  // Won't be shown
}
main()

Program Started
Critical part of the code started.
A cancellation request was received: Timed out waiting for 3000 ms
Critical part of the code encountered an exception.
Finalization has been done.


StandaloneCoroutine{Completed}@53220354

In [11]:
// Or you can use non-cancellable

fun main() = runBlocking {
    launch {
        withTimeoutOrNull(3000) {
            println("Program Started")
            delay(1000)
            criticalPart() 
            delay(1000)  // Cancels here
            println("Program Finished")  // Won't run
        } 
    }
}

suspend fun criticalPart() = withContext(NonCancellable) {
    println("Critical part of the code started.")
    delay(3000)
    println("Critical part of the code finished.") 
}
main()

Program Started
Critical part of the code started.
Critical part of the code finished.


StandaloneCoroutine{Completed}@3e4761ac